In [ ]:
!pip install -q timesfm
!pip -q install huggingface_hub[cli] utilsforecast praxis paxml jax[cuda12]==0.4.26 einshape

In [ ]:
import timesfm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import ast

from datetime import datetime
%matplotlib inline
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
print("Loading data file now, this could take a while depending on file size")
start = time.time()
df = pd.read_csv('/kaggle/input/forecasting-electricity-consumption/train.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")

In [ ]:
tfm = timesfm.TimesFm(
    context_len=192,         # ค่าขนาดข้อมูล input_patch_len * 2 , 3 , 4 , ...
    horizon_len=12,         # จำนวนลำดับที่คาดการณ์ 
    input_patch_len=32,     # ปรับให้เป็นค่าเดียวกับ context_len %fix
    output_patch_len=128,    # ขนาดของแพทช์ข้อมูลที่คาดการณ์ %fix
    num_layers=20,           # จำนวนเลเยอร์ %fix
    model_dims=1280,        # ขนาดมิติของโมเดล %fix
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
def get_season(month):
    if month in [2, 3, 4, 5]:  # ฤดูร้อน
        return 'Summer'
    elif month in [6, 7, 8, 9, 10]:  # ฤดูฝน
        return 'Rainy'
    else:  # ฤดูหนาว
        return 'Winter'
# เพิ่มคอลัมน์เดือนและฤดู
df['ds'] = df['num_id'] % 12
df['Season'] = df['ds'].apply(get_season)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

start_date = pd.to_datetime("2014-01-01")  # เดือน ม.ค. 2557
end_date = pd.to_datetime("2023-06-01")    # เดือน มิ.ย. 2566


data_y1 = pd.DataFrame({
    'ds': pd.date_range(start='2014-01-01', periods=114, freq='M'),
    'y1': df['y1'],
    'unique_id': ['y1'] * 114
})

data_y2 = pd.DataFrame({
    'ds': pd.date_range(start='2014-01-01', periods=114, freq='M'),
    'y2': df['y2'],
    'unique_id': ['y2'] * 114
})

data_y3 = pd.DataFrame({
    'ds': pd.date_range(start='2014-01-01', periods=114, freq='M'),
    'y3': df['y3'],
    'unique_id': ['y3'] * 114
})

In [ ]:
forecast_df_y1 = tfm.forecast_on_df(
    inputs=data_y1,
    freq="M",
    value_name="y1",  # ต้องตรงกับชื่อคอลัมน์
    num_jobs=-1,
)

forecast_df_y2 = tfm.forecast_on_df(
    inputs=data_y2,
    freq="M",
    value_name="y2",  # ต้องตรงกับชื่อคอลัมน์
    num_jobs=-1,
)

forecast_df_y3 = tfm.forecast_on_df(
    inputs=data_y3,
    freq="M",
    value_name="y3",  # ต้องตรงกับชื่อคอลัมน์
    num_jobs=-1,
)

In [ ]:
prediction_y1_np=np.array([forecast_df_y1['timesfm']])
prediction_y2_np=np.array([forecast_df_y2['timesfm']])
prediction_y3_np=np.array([forecast_df_y3['timesfm']])

In [ ]:
prediction_y1_np
prediction_y2_np
prediction_y3_np

In [ ]:
np_original_data_y1 = np.array([[1026340.25,  936407.3 ,  907058.1 ,  883658.6 ,  879586.4 ,
         844402.9 ,  836691.  ,  867292.25, 1197043.2 , 1476412.9 ,
        1575194.6 , 1311701.  ]])

np_original_data_y2 = np.array([[64436072., 66194192., 62646024., 62495128., 60237336., 53534776.,
        54221604., 54082088., 62524664., 55753156., 62791804., 62187372.]])

np_original_data_y3 = np.array([[2138774. , 2025948.2, 2019808.8, 2008996.5, 1867466.8, 1691311.6,
        1497950.1, 1407528.5, 1701985.8, 2090395.8, 2447529. , 2194872.8]])

# Flatten the arrays to 1D
np_original_data_y1 = np_original_data_y1.flatten()
np_original_data_y2 = np_original_data_y2.flatten()
np_original_data_y3 = np_original_data_y3.flatten()

# Combine the data from the three arrays
combined_data = np.concatenate([np_original_data_y1, np_original_data_y2, np_original_data_y3])

# Create an id array from 1 to the length of combined_data
id_array = np.arange(1, len(combined_data) + 1)

# Create a DataFrame
df = pd.DataFrame({
    'id': id_array,
    'Predict': combined_data
})

# Specify the CSV file name
csv_file = 'predictions_1.csv'  # Change this to your desired file name

# Save to CSV
df.to_csv(csv_file, index=False)